# 3.3.3. BioSent2Vec

In [4]:
# load data 
import pandas as pd
df = pd.read_csv('clinic.csv')
df.head()

,Sent1,Sent2,Score
0,Insulin NPH Human [NOVOLIN N] 100 unit / mL su...,Insulin NPH Human [NOVOLIN N] 100 unit / mL su...,3.50
1,"Patient arrives ambulatory, Gait steady, Hist...","Complex assessment performed, Patient arrives...",2.50
2,"Peripheral IV site, established in the right f...","Peripheral IV site, present prior to arrival, ...",3.45
3,No : new confusion or inability to stay aler...,No : new confusion or inability to stay aler...,4.00
4,Spent 15 minutes with the patient and greater ...,"Nurse visit ten minutes, over half of which w...",3.00


In [5]:
# check the size of the data and its class distribution
sent1 = df['Sent1'].tolist()
sent2 = df['Sent2'].tolist()
lables = df['Score'].tolist()


In [6]:
%store -r cleaned1
%store -r cleaned2
%store -r dtree1
%store -r dtree2

# data split.
train_text1 = dtree1[:750]
train_text2 = dtree2[:750]
train_labels =lables[:750]

test_text1 = dtree1[750:]
test_text2 = dtree2[750:]
test_labels =lables[750:]

In [7]:
import sent2vec
from scipy.spatial import distance

In [8]:
#load the BioSentVec model
model_path = "/Users/aswath/PycharmProjects/mfac038/IndividualProject/BioSentVec_PubMed_MIMICIII-bigram_d700.bin"
#Downlaod link -> https://ftp.ncbi.nlm.nih.gov/pub/lu/Suppl/BioSentVec/BioSentVec_PubMed_MIMICIII-bigram_d700.bin

In [9]:
model = sent2vec.Sent2vecModel()
try:
    model.load_model(model_path)
except Exception as e:
    print(e)
print('model successfully loaded')

model successfully loaded


In [10]:
se1=[]
se2=[]
se_t1=[]
se_t2=[]

for i in range(0,len(train_text1)):
    sentence_vector1 = model.embed_sentence(train_text1[i])
    se1.append(sentence_vector1)# sentence vector for train text 1
#print(se1)

for i in range(0,len(train_text2)):
    sentence_vector2 = model.embed_sentence(train_text2[i])
    se2.append(sentence_vector2)# sentence vector for train text 2
# print(se2)

for i in range(0,len(test_text1)):
    sentence_vector3 = model.embed_sentence(test_text1[i])
    se_t1.append(sentence_vector3)# sentence vector for test text 1
# print(se_t1)

for i in range(0,len(test_text2)):
    sentence_vector4 = model.embed_sentence(test_text2[i])
    se_t2.append(sentence_vector4)# sentence vector for test text 2
# print(se_t2)

In [11]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.metrics import mean_squared_error
nsamples1, nx1, ny1 = np.shape(se1)
nsamples2, nx2, ny2 = np.shape(se2)
nsamples3, nx3, ny3 = np.shape(se_t1)
nsamples4, nx4, ny4 = np.shape(se_t2)


se_1=np.array(se1)
se_2=np.array(se2)
se_t_1=np.array(se_t1)
se_t_2=np.array(se_t2)

In [12]:
#Biosentvec + Random Forest
reg1 = RandomForestRegressor(max_depth=6).fit(se_1.reshape((nsamples1,nx1*ny1)), train_labels)
reg2 = RandomForestRegressor(max_depth=6).fit(se_2.reshape((nsamples2,nx2*ny2)), train_labels)

# test model
test_pred1 = reg1.predict(se_t_1.reshape((nsamples3,nx3*ny3)))
test_pred2 = reg2.predict(se_t_2.reshape((nsamples4,nx4*ny4)))

c_mse1 = mean_squared_error(test_labels, test_pred1)
c_mse2 = mean_squared_error(test_labels, test_pred2)

print('MSE for Sentence  1: ', c_mse1)

print("-----------------------------------------")

print('MSE for Sentence  2: ', c_mse2)


MSE for Sentence  1:  1.1016878867505036
-----------------------------------------
MSE for Sentence  2:  1.0396737610570217


#### Cosine Similarity 

In [15]:
#train data
c_cc=[]
for i in range(0,len(train_labels)):
    c= 1 - distance.cosine(se1[i].reshape(1, -1),se2[i].reshape(1, -1))
    c_cc.append(c)
print(c_cc)
len(c_cc)




[0.9723449945449829, 0.8499653339385986, 0.8078193068504333, 0.957718014717102, 0.6277724504470825, 0.6733900308609009, 0.8571213483810425, 0.8040734529495239, 0.2964424788951874, 0.6678501963615417, 0.8996924757957458, 0.38137879967689514, 0.7638356685638428, 0.886290967464447, 0.8628506660461426, 0.9701316952705383, 0.7278345823287964, 0.9643356800079346, 0.9343899488449097, 0.8437682390213013, 0.886152982711792, 0.9290165901184082, 0.9020270705223083, 0.856621503829956, 0.9847961664199829, 0.9056791067123413, 0.38668668270111084, 0.9274587035179138, 0.976249635219574, 0.9927037954330444, 0.9836218357086182, 1.0, 0.8602093458175659, 0.8715192079544067, 0.5495592355728149, 0.8656753301620483, 0.22402870655059814, 0.9082754254341125, 0.6389493942260742, 0.7663381099700928, 0.8507509827613831, 0.3819536566734314, 0.8146626353263855, 0.5140029191970825, 0.28785496950149536, 0.528450071811676, 0.711992084980011, 0.8934186697006226, 0.8841626644134521, 0.9587668180465698, 0.659490287303924

750

In [16]:
#test data
c_cc1=[]
for i in range(0,len(test_labels)):
    c= 1 - distance.cosine(se_t1[i].reshape(1, -1),se_t2[i].reshape(1, -1))
    c_cc1.append(c)
print(c_cc1)
len(c_cc1)


[0.8661053776741028, 0.5638494491577148, 0.8366941213607788, 0.7861780524253845, 0.6826892495155334, 0.9411307573318481, 0.7992367744445801, 0.8617837429046631, 0.8018254041671753, 0.9823338389396667, 0.31612539291381836, 0.5840931534767151, 0.9581778049468994, 0.946699857711792, 0.5446663498878479, 0.9387087821960449, 0.9371199011802673, 0.8639547824859619, 0.7496351003646851, 0.5437052249908447, 0.7000460624694824, 0.5139451622962952, 0.8653411865234375, 0.8409249186515808, 0.6392269134521484, 0.8649019002914429, 0.875474214553833, 0.6626542806625366, 0.6140609979629517, 0.8370217084884644, 0.9110248684883118, 0.9107341766357422, 0.9710434675216675, 0.9320639967918396, 0.855881929397583, 0.9310513734817505, 0.5007864832878113, 0.963945746421814, 0.8662183880805969, 0.8597292304039001, 0.7990186214447021, 0.9168583750724792, 0.8709093928337097, 0.8974343538284302, 0.9768387675285339, 0.9167588353157043, 0.3705829083919525, 0.681548535823822, 0.9131682515144348, 0.9033966064453125, 0.7

318

### Pearson Correlation 

In [17]:

import numpy as np
from scipy.stats import pearsonr    
c_pc, _ = pearsonr(c_cc, train_labels)
print('Pearsons correlation for Train: %.5f' % c_pc)

Pearsons correlation for Train: 0.75764


In [18]:
import numpy as np
from scipy.stats import pearsonr
c_pc_t, _ = pearsonr(c_cc1, test_labels)
print('Pearsons correlation for Test: %.5f' % c_pc_t)

Pearsons correlation for Test: 0.75294


In [19]:
#storing Variables
%store c_mse1
%store c_mse2
%store c_cc
%store c_cc1
%store c_pc
%store c_pc_t

Stored 'c_mse1' (float64)
Stored 'c_mse2' (float64)
Stored 'c_cc' (list)
Stored 'c_cc1' (list)
Stored 'c_pc' (float64)
Stored 'c_pc_t' (float64)
